Loading and Analyzing Data
==========================

In [136]:
import pandas as pd  # programmers like shortening names for things, so they usually import pandas as "pd"
import altair as alt # again with the shortening of names
import requests # we use it for downloading the data so we don't have to save it on GitHub (too big!)
import zipfile # for de-compressing the files we download from the EPA

## Load the air quality data
The EPA published PM2.5 daily summary files annually [on their website](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily). This data is the "PM2.5 FRM/FEM Mass (88101)" dataset.
Pandas understands what a CSV file is, so here we can just load them into two `DataFrame`s. A data frame is simply one representation of a table of data. It is the most important form of storage for working with data in pandas.

In [139]:
# Download the data from the EPA website
data_file_urls = [
    'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2020.zip',
    'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2019.zip'
]
# copied this example from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
for url in data_file_urls:
    local_filename = "data/{}".format(url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
# and unzip the files
files_to_unzip = ["data/{}".format(url.split('/')[-1]) for url in data_file_urls]
for f in files_to_unzip:
    with zipfile.ZipFile(f,"r") as zip_ref:
        zip_ref.extractall("data")

In [140]:
air_2019_df = pd.read_csv("data/daily_88101_2019.csv")
air_2020_df = pd.read_csv("data/daily_88101_2020.csv")
air_2020_df.head() # this helpfully prints out the first few rows with headers to preview the data

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,48.0,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-10-22
1,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,13.0,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-10-22
2,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,14.0,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-10-22
3,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,39.0,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-10-22
4,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,29.0,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2020-10-22


In [21]:
"{} rows for 2019, {} rows for 2020".format(air_2019_df.shape[0], air_2019_df.shape[0])

'539821 rows for 2019, 539821 rows for 2020'

## Aggregate and average MA data by city
Let's compare MA data by city in 2020 and 2019.

In [128]:
# Step 1 - filter the data down by state name

is_MA_data = air_2020_df['State Name'] == "Massachusetts"
air_MA_2020_df = air_2020_df[is_MA_data]

is_MA_data = air_2019_df['State Name'] == "Massachusetts"
air_MA_2019_df = air_2019_df[is_MA_data]

"{} reports for MA in 2019, {} reports for MA in 2020".format(air_MA_2019_df.shape[0], air_MA_2020_df.shape[0])

'11165 reports for MA in 2019, 7105 reports for MA in 2020'

In [129]:
# now trim down to just the columns we care about so it is easier to understand
interesting_columns = ['City Name', 'Latitude', 'Longitude', 'Arithmetic Mean']
air_MA_2020_df = pd.DataFrame(air_MA_2020_df, columns=interesting_columns)
air_MA_2019_df = pd.DataFrame(air_MA_2019_df, columns=interesting_columns)
air_MA_2019_df

,City Name,Latitude,Longitude,Arithmetic Mean
224557,Pittsfield,42.452299,-73.239648,1.0
224558,Pittsfield,42.452299,-73.239648,9.1
224559,Pittsfield,42.452299,-73.239648,3.0
224560,Pittsfield,42.452299,-73.239648,2.5
224561,Pittsfield,42.452299,-73.239648,7.6
...,...,...,...,...
235717,Worcester,42.263955,-71.794322,9.6
235718,Worcester,42.263955,-71.794322,5.9
235719,Worcester,42.263955,-71.794322,9.1
235720,Worcester,42.263955,-71.794322,4.1


In [132]:
# now group all the records by city and average them
avg_by_city_2020_MA_df = air_MA_2020_df.groupby('City Name').mean()\
    .reset_index()\
    .rename(columns={'City Name': 'City', 'Arithmetic Mean': 'Mean'})
avg_by_city_2019_MA_df = air_MA_2019_df.groupby('City Name').mean()\
    .reset_index()\
    .rename(columns={'City Name': 'City', 'Arithmetic Mean': 'Mean'})
# now we need to add in a year column so we can tell the data apart!
avg_by_city_2020_MA_df['year'] = 2020
avg_by_city_2019_MA_df['year'] = 2019
# now we can just contacetane the two dataframes to get all our data in one place
ma_city_avg_df = avg_by_city_2019_MA_df.append(avg_by_city_2020_MA_df)
ma_city_avg_df.to_csv('data/MA-city-year-avg.csv')
ma_city_avg_df

,City,Latitude,Longitude,Mean,year
0,Boston,42.328287,-71.069233,7.180022,2019
1,Brockton,42.065106,-71.012129,6.602431,2019
2,Chelmsford (Chelmsford Center),42.612085,-71.306986,6.952305,2019
3,Chicopee,42.194380,-72.555112,4.739232,2019
4,Fall River,41.685707,-71.169235,6.728876,2019
5,Greenfield,42.605816,-72.596689,6.430217,2019
6,Haverhill,42.770837,-71.102290,5.577212,2019
7,Lynn,42.474642,-70.970816,6.567789,2019
8,North Adams,42.702223,-73.110414,6.211754,2019
9,Pittsfield,42.452158,-73.240124,6.784834,2019


## Visually Inspect the Data

In [131]:
alt.Chart(ma_city_avg_df, height=300).mark_bar().encode(
    alt.X('year:N'),
    alt.Y('Mean'),
    color='year:N',
    column=alt.Column(field='City', type='ordinal', spacing=10)
).properties( 
    title="MA City Average PM2.5 (by year)",
)

alt.Chart(...)